In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import healpy as hp

from pyoperators import MPI

import qubic
from qubic.lib.Qdictionary import qubicDict
from qubic.lib.Calibration.Qselfcal import scatter_plot_FP, get_TEScoordinates_ONAFP
from qubic.lib.Instrument.Qinstrument import QubicInstrument
from qubic.lib.Instrument.Qacquisition import QubicMultiAcquisitions
from qubic.lib.Qsamplings import get_pointing

In [ ]:
nside = 128

# Initialize QUBIC instance

Build the QUBIC dictionary, mandatory to use the QUBIC functions

In [ ]:
def get_dict(comm, key='in'):
    """QUBIC dictionary.

    Method to modify the qubic dictionary.

    Parameters
    ----------
    key : str, optional
        Can be "in" or "out".
        It is used to build respectively the instances to generate the TODs or to reconstruct the sky maps,
        by default "in".

    Returns
    -------
    dict_qubic: dict
        Modified QUBIC dictionary.

    """

    args = {
        "npointings": 1,
        "nf_recon": 2,
        "nf_sub": 20,
        "nside": nside,
        "MultiBand": True,
        "period": 1,
        "RA_center": 0,
        "DEC_center": -57,
        "filter_nu": 150 * 1e9,
        "noiseless": True,
        "comm": comm,
        "dtheta": 15,
        "nprocs_sampling": 1,
        "nprocs_instrument": comm.Get_size(),
        "photon_noise": False,
        "nhwp_angles": 1,
        "effective_duration150": 3,
        "effective_duration220": 3,
        "filter_relative_bandwidth": 0.25,
        "type_instrument": "two",
        "TemperatureAtmosphere150": None,
        "TemperatureAtmosphere220": None,
        "EmissivityAtmosphere150": None,
        "EmissivityAtmosphere220": None,
        "detector_nep": float(4.7e-17),
        "synthbeam_kmax": 2,
        "synthbeam_fraction": 1,
        "beam_shape": "gaussian",
        "debug":False,
        "kind": "IQU",
    }

    ### Get the default dictionary
    dictfilename = "dicts/pipeline_demo.dict"
    dict_qubic = qubicDict()
    dict_qubic.read_from_file(dictfilename)

    for i in args.keys():

        dict_qubic[str(i)] = args[i]

    return dict_qubic

In [ ]:
# Build MPI communicator and qubic dict
comm = MPI.COMM_WORLD
dict_qubic = get_dict(comm)

In [ ]:
# Build scanning strategy
sampling = get_pointing(dict_qubic)

# Build QubicInstrument
q = QubicInstrument(dict_qubic)

In [ ]:
# Number of sub-acquisitions
nsub = 10

In [ ]:
# Initialize the Qubic multi-frequency acquisition
# I multiply by 2 Nsub to have Nsub bands in each band, the rest of the code will only use the first Nsub bands (meaning the ones in the 150 GHz band)
acq = QubicMultiAcquisitions(dict_qubic, nsub*2, 2, sampling=sampling)

# Build a list that contains all sub-ProjectionOperators, meaning the ProjectionOperators for each sub-bands
Proj_list = []
for i in range(nsub):
    Proj_list.append(acq.subacqs[i].get_projection_operator())

# Build TOD

In [ ]:
# Create uniform maps of "1"
input_maps = np.ones((nsub, hp.nside2npix(nside), 3))

# Build the list of sub-TODs
tod = []
for i in range(nsub):
    tod.append(Proj_list[i](input_maps[i]))

In [ ]:
print("Number of sub-TODs: ", len(tod))
print("sub-TODs shape: ", tod[0].shape, " = (Ndet, Nsample, Nstk)")

# Relative difference between succesive sub-TODs

## Mean on detectors

In [ ]:
# Intensity
plt.figure()
for i in range(nsub-1):
    diff = np.abs((tod[i+1][:, 0, 0] - tod[i][:, 0, 0])/tod[i][:, 0, 0])
    mean_diff = np.mean(diff)
    plt.plot(acq.allnus[i], mean_diff, '.')
    
plt.title(r"$H = \mathcal{P}_{roj}$ - Intensity map")
plt.ylabel(r'$\frac{| H(Imap)^{i+1} - H(Imap)^i |}{| H(Imap)^{i} |}$', fontsize=15, labelpad=10)
plt.xlabel('Frequency (GHz)')

In [ ]:
# Polarisation
plt.figure()
for i in range(nsub-1):
    diff = np.abs((tod[i+1][:, 0, 1] - tod[i][:, 0, 1])/tod[i][:, 0, 1])
    mean_diff = np.mean(diff)
    plt.plot(acq.allnus[i], mean_diff, '.')
    
plt.title(r"$H = \mathcal{P}_{roj}$ - Q map")
plt.ylabel(r'$\frac{| H(Qmap)^{i+1} - H(Qmap)^i |}{| H(Qmap)^{i} |}$', fontsize=15, labelpad=10)
plt.xlabel('Frequency (GHz)')

## For each detectors

In [ ]:
plt.figure()

# Range of detectors for which we want to plot the difference
idet_min, idet_max = 200, 210

for idet in range(idet_min, idet_max):
    for i in range(nsub-1):
        diff = (tod[i+1][idet, 0, 1] - tod[i][idet, 0, 1])/tod[i][idet, 0, 1] #np.abs((tod[i+1][idet, 0, 1] - tod[i][idet, 0, 1])/tod[i][idet, 0, 1])
        plt.plot(acq.allnus[i], diff, '.b')
        
    plt.title(r"$H = \mathcal{P}_{roj}$ - Q map - " + f'Detector index = {idet}')
    plt.ylabel(r'$\frac{ H(Qmap)^{i+1} - H(Qmap)^i }{ H(Qmap)^{i}}$', fontsize=15, labelpad=10)
    plt.xlabel('Frequency (GHz)')
    plt.show()

# Relative difference on the focal plane

Plot the difference between two successive tods, at a given time, on the focal plane. The polarisation case seems very strange with respect to the intensity case.

In [ ]:
# Compute the position of the TES in the focal plane
xTES, yTES, vertex = get_TEScoordinates_ONAFP(q)

In [ ]:
# Sub-bands indices for the comparison
ind_a, ind_b = 3, 2

# Intensity
fig, ax = plt.subplots(figsize=(10, 10))
relative_diff = (tod[ind_a]-tod[ind_b])/tod[ind_b]*100
print(relative_diff.shape)
scatter_plot_FP(q, xTES, yTES, np.mean(relative_diff[...,0], axis=1), frame='ONAFP', s=150, title=r'Intensity map - $\frac{ \mathcal{P}_{roj}(Imap)^{n+1} - \mathcal{P}_{roj}(Imap)^{n} }{ \mathcal{P}_{roj}(Imap)^{n} } x 100$', fig=fig, ax = ax, unit='%')

In [ ]:
# Polarisation Q
fig, ax = plt.subplots(figsize=(10, 10))
relative_diff = (tod[ind_a]-tod[ind_b])/tod[ind_b]*100
scatter_plot_FP(q, xTES, yTES, relative_diff[:, 0, 1], frame='ONAFP', s=150, title=r'Q map - $\frac{ \mathcal{P}_{roj}(Qmap)^{n+1} - \mathcal{P}_{roj}(Qmap)^{n} }{ \mathcal{P}_{roj}(Qmap)^{n} } x 100$', fig=fig, ax = ax, unit='%')

In [ ]:
# Polarisation U
fig, ax = plt.subplots(figsize=(10, 10))
relative_diff = (tod[ind_a]-tod[ind_b])/tod[ind_b]*100
scatter_plot_FP(q, xTES, yTES, relative_diff[:, 0, 2], frame='ONAFP', s=150, title=r'U map - $\frac{ \mathcal{P}_{roj}(Umap)^{n+1} - \mathcal{P}_{roj}(Umap)^{n} }{ \mathcal{P}_{roj}(Umap)^{n} } x 100$', fig=fig, ax = ax, unit='%')

# Peak Amplitude evolution with frequency

P.matrix.data is build such that you have all the points for the same detector at all the different time samples, then the next detector at all the different time samples, etc.

In [ ]:
# Indices to compute the amplitude of peaks
peak_index = 0
N_sample = 0
time_index = 0
det_index = 23

# Plot the amplitude of the peak for each sub-band
for peak_index in range((2*dict_qubic['synthbeam_kmax']+1)**2):
    plt.figure()
    peak_amplitude = []
    for i in range(nsub):
        peak_amplitude.append(Proj_list[i].matrix.data.r11[(det_index*N_sample)+time_index, peak_index])

    plt.plot(acq.allnus[:nsub], peak_amplitude)
    plt.xlabel("Frequency (GHz)")
    plt.ylabel("Peak Amplitude")
    plt.title(f"Detector Index = {det_index} | Time Sample Index = {time_index} | Peak Index = {peak_index}")
    plt.show()

# Comparing Primary and Synthesized Beams evolution with frequency

In [ ]:
# Indices for the plot
idet = 0
N_sample = 0
time_index = 0

# Store the primary beam for each sub-band
primbeam_frequencies = [acq.multiinstrument[i].primary_beam for i in range(nsub)]

for ipeak in range((2*dict_qubic['synthbeam_kmax']+1)**2):
    peak_amplitude = [] 
    fig, ax1 = plt.subplots()
    for i in range(nsub):
        # Sub-acquisition for the i-th sub-band
        subacq_i = acq.subacqs[i]

        # Compute the peaks position and ampplitude
        th, ph, v = subacq_i.instrument._peak_angles(acq.scene, acq.allnus[i]*1e9, 
                                        subacq_i.instrument.detector.center,
                                        subacq_i.instrument.synthbeam,
                                        subacq_i.instrument.horn,
                                        subacq_i.instrument.primary_beam)

        # Plot the amplitude of the primary beam at peaks' position
        ax1.plot(acq.allnus[i], primbeam_frequencies[i](th[idet, ipeak], ph[idet, ipeak]), '.b')
        
        # Store the amplitude of the peak
        # Note: I checked that both gives the same result 
        peak_amplitude.append(v[idet, ipeak])
        #peak_amplitude.append(Proj_list[i].matrix.data.r11[(idet*N_sample)+time_index, ipeak])

    ax1.set_xlabel('Frequency (GHz)')
    ax1.set_ylabel('Primary beam', color = 'blue')
    ax1.set_title('Primary beam and peak amplitude at peak %d for detector %d' % (ipeak, idet))

    ax2 = ax1.twinx()
    ax2.plot(acq.allnus[:nsub], peak_amplitude, '.r')
    ax2.set_ylabel("Peak Amplitude", color = 'r')
    plt.show()